In [ ]:
# Get your keys at developer.twitter.com
# And load them into file

import json

# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = 'your_consumer_key'
credentials['CONSUMER_SECRET'] = 'your_consumer_secret'
credentials['ACCESS_TOKEN'] = 'your_access_token'
credentials['ACCESS_SECRET'] = 'your_access_secret'

# Save the credentials object to file
with open('twitter_credentials.json', 'w') as file:
    json.dump(credentials, file)


In [132]:
# Import the Twython class
from twython import Twython
import json

with open('twitter_credentials.json', 'r') as file:
    creds = json.load(file)

python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

import re
# emoji_pattern = re.compile("["
#     u"\U0001F600-\U0001F64F"  # emoticons
#     u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#     u"\U0001F680-\U0001F6FF"  # transport & map symbols
#     u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#     u"\U00002702-\U000027B0"
#     u"\U000024C2-\U0001F251"
#     "]+", flags=re.UNICODE)

# clean tweet text from links, mentions, etc
def tweet_cleaner(tweet_text):
    striped = tweet_text.strip()
    filtered_links = re.sub(r'http\S+', '', striped)
    filtered_mentions = re.sub(r'@\S+', '', filtered_links)
    # filtered_emoji = emoji_pattern.sub(r'', filtered_mentions)
    removed_dup_spaces = ' '.join(filtered_mentions.split())
    return removed_dup_spaces
# result = tweet_cleaner('  Виснет 😩 https://t.co/9GMRq47kdJ   @UkrainianMonbe1 @Nat__Mel 12')
# print(result)

# validation of tweet text
def tweet_validator(tweet_text, min_len = 30, with_rt = False):
    is_valid = True
    not_valid = False

    if (not with_rt and tweet_text.startswith('RT')):
        return not_valid
    if (len(tweet_text) < min_len):
        return not_valid
    return is_valid

def get_tweets_dict(query, number_of_items):
    dict_ = {
        # 'user': [],
        # 'date': [],
        'text': [],
        'query': [],
        # 'favorite_count': []
    }

    query_q = query.get('q')
    results = python_tweets.cursor(python_tweets.search, **query)

    for result in results:
        result_text = tweet_cleaner(result['text'])
        if (not tweet_validator(result_text)):
            continue

        # dict_['user'].append(result['user']['screen_name'])
        # dict_['date'].append(result['created_at'])
        dict_['text'].append(result_text)
        dict_['query'].append(query_q)
        # dict_['favorite_count'].append(result['favorite_count'])

        if (len(dict_['text']) >= number_of_items):
            return dict_
    return dict_


# Structure data in a pandas DataFrame for easier manipulation
# import pandas as pd
# query = { 'q': 'some query', 'lang': 'uk' }
# df = pd.DataFrame(get_tweets_dict(query, 2))
# df.sort_values(by='favorite_count', inplace = True, ascending = False)
# print(df)


In [137]:
def write_df_to_json(df, file_path):
    # import json
    with open(file_path, 'w', encoding = 'utf-8') as file:
        df.to_json(file, orient = 'records', force_ascii = False)

query = {
    'q': '😩',
    # 'result_type': 'popular',
    'lang': 'uk',
}

df = pd.DataFrame(get_tweets_dict(query, 10))
# df.sort_values(by='favorite_count', inplace = True, ascending = False)
# print(df)
write_df_to_json(df, 'data/twitter-data-2.json')

In [145]:
def load_labeled_tweets(queries, isPositive, common_dict):
    for query_q in queries:
        query = { 'q': query_q, 'lang': 'uk' }
        tweets_dict = get_tweets_dict(query, 100)

        tweets_dict_len = len(tweets_dict['text'])
        common_dict['text'] += tweets_dict['text']
        common_dict['query'] += tweets_dict['query']
        common_dict['isPositive'] += [isPositive] * tweets_dict_len
        print (query)
        
common_dict = {
    'text': [],
    'query': [],
    'isPositive': [],
}

positive_queries = [
    'добре',
    'добрий',
    '😊',
    '😘',
]

negative_queries = [
    'погано',
    'поганий',
    '😭',
    '😡',
]

load_labeled_tweets(positive_queries, True, common_dict)
load_labeled_tweets(negative_queries, False, common_dict)

df = pd.DataFrame(common_dict)

number_of_rows = df.shape[0]
print('Number of rows', number_of_rows)
write_df_to_json(df, 'data/twitter-data-3.json')

{'q': 'добре', 'lang': 'uk'}
{'q': 'добрий', 'lang': 'uk'}
{'q': '😊', 'lang': 'uk'}
{'q': '😘', 'lang': 'uk'}
{'q': 'погано', 'lang': 'uk'}
{'q': 'поганий', 'lang': 'uk'}
{'q': '😭', 'lang': 'uk'}
{'q': '😡', 'lang': 'uk'}
8
